In [ ]:
from dotenv import load_dotenv
import openai
import os
from pathlib import Path  # Python 3.6+ only
import json

env_path = Path('../') / 'secrets.env'
load_dotenv(dotenv_path=env_path)
key = os.getenv("AZURE_OPENAI_API_KEY")
openai_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
openai.api_type = "azure"
openai.api_version = "2023-05-15" 
openai.api_key= key
openai.api_base=openai_endpoint
engine = "gpt-35-turbo"

In [ ]:
user_message = ""
def generate_categories():
    user_message =f""" 
    Think of 40 main categories of customer support conversation to classify customer support calls.
    Output data into JSON format .
    Here is the generated data in JSON format:
    
    [{{"category name place_holder":"description place holder"}},{{"category name place holder":"description place holder"}}]
 
"""

    response = openai.ChatCompletion.create(
        engine=engine, # engine = "deployment_name".
        messages=[
            {"role": "system", "content": "You are a customer support data analyst"},
            {"role": "user", "content":user_message },
        ]
    )
    return response['choices'][0]['message']['content']
n=0
categories=""
while (n<5):
    try:

        categories = generate_categories()

        categories = json.loads(categories)
        break
    except:
        n+=1

if len(categories)==0:
    print("data generation fails, please try again")


In [ ]:
import ast
user_message = ""
def generate_sub_categories(category):
    user_message =f""" 
    Given this customer support call category \n {category}",\n think of 10-20 subcategories under the category that can further classify the call content.
    Output the data into a list of elements where each element is a tuple of name and description.
    Follow this format template for the output [("name","description"),("name","description")]
    Here are the subcategories for the given category:
    
 
"""

    response = openai.ChatCompletion.create(
        engine=engine, # engine = "deployment_name".
        messages=[
            {"role": "system", "content": "You are a customer support data analyst"},
            {"role": "user", "content":user_message },
        ]
    )
    return response['choices'][0]['message']['content']
subcagories={}
for category in categories['categories']:
    
    n=0
    while (n<2):
        try:

            sub_cat = generate_sub_categories(category)
            sub_cat = ast.literal_eval(sub_cat)
            assert type(sub_cat)==list
            assert type(sub_cat[0])==tuple
        
            break
        except:
            n+=1
    subcagories[list(category)[0]]=(list(category.values())[0], sub_cat)


In [ ]:
import time
def generate_call(description):
    user_message =f""" 
    given {description} as the main topic of an customer support call, generate content of the call.
    The content is: 
 
"""
    response = openai.ChatCompletion.create(
        engine="gpt-35-turbo", # engine = "deployment_name".
        messages=[
            {"role": "system", "content": "You are a helpful AI assistant that help generate content of customer support call transcript"},
            {"role": "user", "content":user_message },
        ]
    )
    return response['choices'][0]['message']['content']

contents =[]
for category in categories['categories']:

    for subcategory in subcagories[list(category)[0]][1]:
        description = f"""{list(category)[0]}\n
                        {subcategory[1]}"""
        n=0
        while(n<3):
            try:
                content = (generate_call(description), list(category)[0], subcategory[0])
                break
            except:
                n+=1
                time.sleep(3)
        contents.append(content)
folder_path ="data"
os.makedirs(folder_path, exist_ok=True)
file_name = f"contents.txt"
with open(os.path.join(folder_path,file_name), 'w') as file:
    file.write(json.dumps(contents))


In [ ]:
import time
def generate_kb(description):
    user_message =f""" 
    given {description} as the main topic of an customer support call, generate a knowledge article as guidance to handle the situation.
    The content is: 
 
"""
    response = openai.ChatCompletion.create(
        engine="gpt-35-turbo", # engine = "deployment_name".
        messages=[
            {"role": "system", "content": "You are a helpful AI assistant that help generate content of knowledge articles"},
            {"role": "user", "content":user_message },
        ]
    )
    return response['choices'][0]['message']['content']

kb_contents =[]
for category in categories:

    for subcategory in subcagories[category[0]]:
        description = f"""{category[0]}\n
                        {subcategory[1]}"""
        n=0
        while(n<3):
            try:
                kb_content = (generate_kb(description), category[0], subcategory[0])
                break
            except:
                n+=1
                time.sleep(3)
        kb_contents.append(kb_content)
folder_path ="data"
os.makedirs(folder_path, exist_ok=True)
file_name = f"kb_contents.txt"
with open(os.path.join(folder_path,file_name), 'w') as file:
    file.write(json.dumps(kb_contents))
